
## Building a Conversational Chatbot with Custom Data 

This notebook guides you through the creation of a chatbot tailored to your specific data needs. Utilizing HuggingFaceEmbeddings and FAISS, the project transforms documents into vectors for a local vector storage system. Then it integrates the "meta-llama/llama-2-7b-chat" model from your local machine. The `langchain` library plays a crucial role in this process, aiding in tasks like chunking documents, indexing data in vector db, managing conversation chains with memory buffers, and crafting prompt templates.


### Key Features:

- **PDF Content Processing**: When users upload PDF files, the notebook extracts the text, segments it into manageable chunks, and indexes these chunks in in a vector db locally using HuggingFaceEmbeddings and FAISS.
- **Data-Driven Query Handling**: Users can pose questions to the chatbot, which searches the indexed data for relevant answers.
- **Integrating Vector Database and LLMs**: We leverage `langchain`'s capabilities to link vector database indexing with llama-2 LLMs, enabling a seamless conversational experience with memory and retrieval functionalities.
- **Hallucination Check**: The notebook includes a mechanism to detect and correct any hallucinations or inaccuracies in the LLM's responses.

### Prerequisites for Running the Notebook:


1. **Library Requirements**: Confirm that you have installed all libraries specified in the `requirements (local rag).txt` file by `pip install -r requirements (local rag).txt`




Below cell imports the required libraries to run this notebook.

In [1]:

import PyPDF2

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp


from langchain.embeddings import HuggingFaceEmbeddings # import hf embedding
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler



### Enter your pdf file name below


In [4]:
pdf_docs=["Vikram Bhat - Experienced Data Scientist.pdf"]

### Step 1: Prepare above documents and their metadata
The prepare_docs function below processes a list of PDF documents by extracting text from each page and organizing it into two lists: one for the text content and another for the metadata (titles). It iterates through each page of each PDF, extracts the text, and forms a title using the PDF name and page number. The function returns these two lists, making it useful for indexing and referencing the content of multiple PDFs at a page level.

In [5]:

def prepare_docs(pdf_docs):
    docs = []
    metadata = []
    content = []

    for pdf in pdf_docs:

        pdf_reader = PyPDF2.PdfReader(pdf)
        for index, text in enumerate(pdf_reader.pages):
            doc_page = {'title': pdf + " page " + str(index + 1),
                        'content': pdf_reader.pages[index].extract_text()}
            docs.append(doc_page)
    for doc in docs:
        content.append(doc["content"])
        metadata.append({
            "title": doc["title"]
        })
    print("Content and metadata are extracted from the documents")
    return content, metadata



### Step 2: Chunk the documents 
The get_text_chunks function takes text content and metadata as inputs and splits the content into smaller chunks. It uses a RecursiveCharacterTextSplitter configured with a specified chunk size (512 characters) and overlap (256 characters) for this purpose. The function processes the content, splitting it into passages while maintaining associated metadata. After splitting, it prints the total number of passages created and returns these split documents. This function is useful for breaking down large text into more manageable, indexed segments for easier processing and retrieval.

In [6]:
def get_text_chunks(content, metadata):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,
        chunk_overlap=256,
    )
    split_docs = text_splitter.create_documents(content, metadatas=metadata)
    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs


### Step 3: Ingest into Vector Database locally

The `ingest_into_vectordb` function is designed for processing and indexing a collection of documents into a vector database using FAISS (Facebook AI Similarity Search) for efficient similarity searches. It operates as follows:

1. **Embedding Creation**: It generates embeddings for the input documents (`split_docs`) using the Hugging Face model `'sentence-transformers/all-MiniLM-L6-v2'`. This model is specifically chosen for its efficiency in creating sentence-level embeddings and is set to run on the CPU.

2. **Vector Database Indexing**: Utilizes the generated embeddings to create a FAISS vector database. FAISS is used for its ability to efficiently handle large-scale similarity searches and clustering of dense vectors.

3. **Local Storage**: After creating the vector database, the function saves it locally to the path specified by `DB_FAISS_PATH`, ensuring the data can be easily accessed for future similarity searches or retrieval tasks.

The primary purpose of this function is to transform textual data into a structured, searchable vector format, facilitating efficient and scalable retrieval tasks such as document similarity searches or clustering.

In [7]:
def ingest_into_vectordb(split_docs):
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
    db = FAISS.from_documents(split_docs, embeddings)

    DB_FAISS_PATH = 'vectorstore/db_faiss'
    db.save_local(DB_FAISS_PATH)
    return db

### Step 4: Set up Conversation Chain using LLM
The `get_conversation_chain` function is designed to create and configure a conversational chain for a language model, specifically using the LLaMA model and a vector database for retrievals. Here's a summary of its main components and functionalities:

1. **Callback Manager Setup**:
   - Initializes a `CallbackManager` with `StreamingStdOutCallbackHandler()`, which likely handles streaming and logging outputs during the model's operation.

2. **LLaMA Model Configuration**:
   - Instantiates a `LlamaCpp` model with specified parameters such as `model_path`, `temperature`, `max_tokens`, `top_p`, and `n_ctx`. These parameters configure the behavior of the LLaMA model, including its conversational style and technical constraints.
   - Integrates the `callback_manager` with the LLaMA model, allowing for additional processing or logging during the model's operation.

3. **Retriever Initialization**:
   - Transforms the input `vectordb` into a retriever, enabling it to fetch relevant information from the vector database during conversations.

4. **Conversation Chain Creation**:
   - Sets up a `ConversationBufferMemory`, which manages the conversation history and assists in generating context-aware responses.
   - Constructs a `ConversationalRetrievalChain` using the LLaMA model (`llama_llm`), the retriever, and the conversation memory. This chain is responsible for handling the flow of the conversation, including retrieving relevant information and generating responses.

5. **Return Value**:
   - Outputs a message indicating the successful creation of the conversational chain.
   - Returns the `conversation_chain` object, which can be used to handle conversational interactions using the LLaMA model and the vector database.

This function sets up a sophisticated conversational AI system combining the LLaMA model for language generation and a vector database for information retrieval, enhanced with a callback manager for additional processing and a conversation memory buffer for context management.

In [8]:
template = """[INST]
As an AI, provide accurate and relevant information based on the provided document. Your responses should adhere to the following guidelines:
- Answer the question based on the provided documents.
- Be direct and factual, limited to 50 words and 2-3 sentences. Begin your response without using introductory phrases like yes, no etc.
- Maintain an ethical and unbiased tone, avoiding harmful or offensive content.
- If the document does not contain relevant information, state "I cannot provide an answer based on the provided document."
- Avoid using confirmatory phrases like "Yes, you are correct" or any similar validation in your responses.
- Do not fabricate information or include questions in your responses.
- do not prompt to select answers. do not ask me questions
{question}
[/INST]
"""

#template = """Given the document and the current conversation between a user and an agent, your task is as follows: Answer any user query by using information from the document. The response should be detailed."""
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
def get_conversation_chain(vectordb):
    llama_llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=200,
    top_p=1,
    callback_manager=callback_manager,
    n_ctx=3000)

    retriever = vectordb.as_retriever()
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(template)

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')

    conversation_chain = (ConversationalRetrievalChain.from_llm
                          (llm=llama_llm,
                           retriever=retriever,
                           #condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                           memory=memory,
                           return_source_documents=True))
    print("Conversational Chain created for the LLM using the vector store")
    return conversation_chain


### Step 5: Detect Hallucination in the LLMs Response
The `validate_answer_against_sources` function evaluates the reliability of a response by comparing it with source documents. It works as follows:

1. **Model Initialization**: Utilizes the SentenceTransformer model 'all-MiniLM-L6-v2' to generate embeddings.

2. **Threshold Setting**: Sets a similarity threshold (here, 0.5) to determine the acceptable level of similarity between the response and source documents.

3. **Extracting Source Texts**: Gathers the content of the source documents.

4. **Computing Embeddings**: Generates embeddings for both the response answer and the source texts.

5. **Calculating Similarity**: Computes cosine similarity scores between the response answer's embedding and the embeddings of each source text.

6. **Validity Check**: Checks if any of the similarity scores exceed the set threshold. If yes, it implies that the response is sufficiently similar to at least one of the source documents, suggesting its reliability, and returns `True`. If not, it returns `False`.

Essentially, this function serves as a mechanism to check the alignment of the chatbot's response with the information in the source documents, ensuring the response's accuracy and relevance.

In [9]:

def validate_answer_against_sources(response_answer, source_documents):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    similarity_threshold = 0.5  
    source_texts = [doc.page_content for doc in source_documents]

    answer_embedding = model.encode(response_answer, convert_to_tensor=True)
    source_embeddings = model.encode(source_texts, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(answer_embedding, source_embeddings)


    if any(score.item() > similarity_threshold for score in cosine_scores[0]):
        return True  

    return False  


Now that we have crafted all the necessary functions, it's time to put them into action and test their functionality.

In [10]:
content, metadata = prepare_docs(pdf_docs)


Content and metadata are extracted from the documents


In [11]:
split_docs = get_text_chunks(content, metadata)

Documents are split into 3 passages


In [ ]:
vectordb=ingest_into_vectordb(split_docs)

In [12]:
conversation_chain=get_conversation_chain(vectordb)

Conversational Chain created for the LLM using the vector store


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

### Ask your Question

We created a conversational chain and now ready to chat with your own data. 


### Question 1

In [14]:
user_question = "who is vikram bhat?"
response=conversation_chain({"question": user_question})
print("Q: ",user_question)
print("A: ",response['answer'])

Llama.generate: prefix-match hit


 Vikram Bhat is a data scientist with expertise in data science and analytics, machine learning, and data visualization. He has experience working in various industries, including finance and healthcare, and has developed end-to-end pipelines for building machine learning models using Watson Studio and AWS Sagemaker.Q:  who is vikram bhat?
A:   Vikram Bhat is a data scientist with expertise in data science and analytics, machine learning, and data visualization. He has experience working in various industries, including finance and healthcare, and has developed end-to-end pipelines for building machine learning models using Watson Studio and AWS Sagemaker.



llama_print_timings:        load time =     429.30 ms
llama_print_timings:      sample time =       5.85 ms /    68 runs   (    0.09 ms per token, 11615.99 tokens per second)
llama_print_timings: prompt eval time =   68382.68 ms /  1435 tokens (   47.65 ms per token,    20.98 tokens per second)
llama_print_timings:        eval time =    4554.99 ms /    67 runs   (   67.98 ms per token,    14.71 tokens per second)
llama_print_timings:       total time =   73274.66 ms /  1502 tokens


We have now received an answer for a provided question. We can also view the conversation history and source documents in the response.


### Question 2

In [15]:
user_question = "where did he graduate?"
response=conversation_chain({"question": user_question})
print("Q: ",user_question)
print("A: ",response['answer'])
print("\nConversation Chain: \n",response)

Llama.generate: prefix-match hit


 Where did Vikram Bhat graduate from?


llama_print_timings:        load time =     429.30 ms
llama_print_timings:      sample time =       0.97 ms /    11 runs   (    0.09 ms per token, 11305.24 tokens per second)
llama_print_timings: prompt eval time =    5938.79 ms /   136 tokens (   43.67 ms per token,    22.90 tokens per second)
llama_print_timings:        eval time =     573.55 ms /    11 runs   (   52.14 ms per token,    19.18 tokens per second)
llama_print_timings:       total time =    6551.86 ms /   147 tokens
Llama.generate: prefix-match hit


 Based on the provided information, Vikram Bhat graduated from University College Cork with an MSc in Data Science and Analytics.Q:  where did he graduate?
A:   Based on the provided information, Vikram Bhat graduated from University College Cork with an MSc in Data Science and Analytics.

Conversation Chain: 
 {'question': 'where did he graduate?', 'chat_history': [HumanMessage(content='who is vikram bhat?'), AIMessage(content=' Vikram Bhat is a data scientist with expertise in data science and analytics, machine learning, and data visualization. He has experience working in various industries, including finance and healthcare, and has developed end-to-end pipelines for building machine learning models using Watson Studio and AWS Sagemaker.'), HumanMessage(content='where did he graduate?'), AIMessage(content=' Based on the provided information, Vikram Bhat graduated from University College Cork with an MSc in Data Science and Analytics.')], 'answer': ' Based on the provided informatio


llama_print_timings:        load time =     429.30 ms
llama_print_timings:      sample time =       2.32 ms /    28 runs   (    0.08 ms per token, 12058.57 tokens per second)
llama_print_timings: prompt eval time =   66654.66 ms /  1482 tokens (   44.98 ms per token,    22.23 tokens per second)
llama_print_timings:        eval time =    1794.07 ms /    27 runs   (   66.45 ms per token,    15.05 tokens per second)
llama_print_timings:       total time =   68707.06 ms /  1509 tokens


### Detect and Solve Hallucinations

The response to the second question appears to be generated by the LLMs and not directly retrieved from the documents, resulting in an answer that seems out of context. To address such instances of misinformation or 'hallucination,' we previously developed the function `validate_answer_against_sources`. We can use this function to cross-check the answer with the source documents to ensure its accuracy and relevance.

In [23]:
if response['source_documents']:
    response_answer = response['answer']
    source_docs = response['source_documents']

    # Post-processing step to validate the answer against the source documents
    is_valid_answer = validate_answer_against_sources(response_answer, source_docs)
    if not is_valid_answer:
        response['answer'] = "Sorry I can not answer the question based on the given documents"
else:
    response['answer'] ="Sorry, I cannot answer the question based on the given documents"

print("Q: ",user_question)
print("A: ",response['answer'])

Q:  where can i find them?
A:    You can find IBM's industry accelerators for different industries by visiting the IBM Cloud Pak for Data documentation website and navigating to the "Integrations" section, where you will find links to the various industry accelerators available. From there, you can download the accelerator that best fits your needs and use it to build and deploy machine learning models tailored to your specific industry.


We have now set up end to end Retrieval Augmented Generation Chatbot using LangChain and Llama 2. 

In [25]:
import panel as pn
pn.extension()



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
# Text input for user's question
question_input = pn.widgets.TextInput(placeholder='Type your question here...', name='Ask a Question')

# Button to submit the question
submit_button = pn.widgets.Button(name='Submit', button_type='primary')

# Area to display conversation history
conversation_history = pn.pane.Markdown("**Conversation History:**\n", width=500, height=200, style={'white-space': 'pre-wrap', 'overflow-y': 'auto'})


/var/folders/l3/524d7s611rs3hl7hsd6rk25w0000gn/T/ipykernel_32547/938895175.py:8: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  conversation_history = pn.pane.Markdown("**Conversation History:**\n", width=500, height=200, style={'white-space': 'pre-wrap', 'overflow-y': 'auto'})


In [27]:
def on_submit(event):
    user_question = question_input.value
    response = conversation_chain({"question": user_question})  # Assuming conversation_chain is defined

    # Update conversation history
    new_entry = f"**Q**: {user_question}\n**A**: {response['answer']}\n\n---\n\n"
    conversation_history.object = new_entry + conversation_history.object

    # Clear the input box
    question_input.value = ''

submit_button.on_click(on_submit)


Watcher(inst=Button(button_type='primary', name='Submit'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_submit at 0x4202f35e0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [28]:
chat_interface = pn.Column(
    "# Chatbot Interface",
    question_input,
    submit_button,
    conversation_history
)

chat_interface.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'511354c4-b7e7-4b14-8451-4e2894d6763a': {'version…

In [18]:
chat_history = []
def on_submit_button_clicked(b):
    with output:
        clear_output()
        user_question = user_question_input.value
        response = conversation_chain({"question": user_question})
        
        # Append to chat history
        chat_history.append(f"Q: {user_question}")
        chat_history.append(f"A: {response['answer']}")

        # Display chat history
        for entry in chat_history:
            print(entry)



In [20]:
from ipywidgets import widgets

# Text input for the user's question
user_question_input = widgets.Text(
    placeholder='Type your question here...',
    description='Question:',
    disabled=False
)

# Button to submit the question
submit_button = widgets.Button(
    description='Ask',
    button_style='info',
    tooltip='Ask the chatbot',
)

# Output area for the chatbot's response
output = widgets.Output()


In [21]:
display(user_question_input, submit_button, output)


Text(value='', description='Question:', placeholder='Type your question here...')

Button(button_style='info', description='Ask', style=ButtonStyle(), tooltip='Ask the chatbot')

Output()